# 気象庁から天気データの所得
気象庁の日ごとの天気をスクレイピングにて所得する

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

place_codeA = [44] #都道府県コード
place_codeB = [47662] #地域コード
place_name = ["東京"]

In [ ]:
# URL
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=1&view=p1"

天気概要を変更する

In [ ]:
unused_weather=["一時","時々","後","、"]
weather_list=["晴","曇","雨","雪"]
def change_weather(weather_conditions):
  txt=str(weather_conditions)
  
  for i in range(4):
    if(txt.find(unused_weather[i])>=0):
      txt=txt[:txt.find(unused_weather[i])]
  for i in range(4):
    if(txt.find(weather_list[i])>=0):
      txt=weather_list[i]
  if(txt.find("みぞれ")>=0):
      txt=weather_list[3]
  
  return txt

In [ ]:
for place in place_name:
  All_list = [["年月日","天気（昼）","天気（夜）"]]
  index = place_name.index(place)

  weather_set={"晴"}
  # for文で該当期間抽出
  for year in range(2020,2023):
    for month in range(1,13):
      r = requests.get(base_url%(place_codeA[index], place_codeB[index], year, month))
      r.encoding = r.apparent_encoding

      soup = BeautifulSoup(r.text)
      rows = soup.findAll('tr',class_='mtx')
      rows = rows[4:]

      for row in rows:
        # trのなかのtdをすべて抜き出す
        data = row.findAll('td')


        rowData = [] #初期化

        #日付を所得
        rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))

        #天気概況を所得
        weather_set.add(change_weather(data[19].string))
        weather_set.add(change_weather(data[20].string))
        rowData.append(change_weather(data[19].string)) 
        rowData.append(change_weather(data[20].string)) 
        All_list.append(rowData)

  #東京.csvを作成
  with open('東京.csv', 'w',encoding="utf_8_sig") as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(All_list)
  print(weather_set)

{'曇', 'None', '雨', '雪', '晴'}


# ついにきた！マルコフ連鎖


1.   昼のみのデータで確率を求める
2.   昼と夜を合わせたデータで確率を求める(todo)



In [ ]:
import numpy as np
with open('東京.csv') as f:
  reader = csv.reader(f)
  data_count=0
  #晴=0,曇=1,雨=2,雪=3とした二次元配列を作成する
  weather_markov=np.zeros((4, 4))
  tmp=-1
  for row in reader:
    data_count+=1
    if(row[1]=='None'):
      break
    weather_now=-1
    #キモイ変換
    if(row[1]=='晴'):
      weather_now=0
    if(row[1]=='曇'):
      weather_now=1
    if(row[1]=='雨'):
      weather_now=2
    if(row[1]=='雪'):
      weather_now=3
    if(tmp!=-1):
      weather_markov[tmp][weather_now]+=1
    if(weather_now!=-1):
      tmp=weather_now
  for i in range(4):
    sum=0
    for j in range(4):
      sum+=weather_markov[i][j]
    for j in range(4):
      weather_markov[i][j]/=sum
    
  print(data_count)
  print(weather_markov)

885
[[0.60246914 0.3654321  0.02962963 0.00246914]
 [0.33692722 0.47708895 0.18598383 0.        ]
 [0.32380952 0.44761905 0.22857143 0.        ]
 [1.         0.         0.         0.        ]]


↑だと見づらいので状態遷移図を作成します

In [ ]:
!pip install transitions
from transitions import Machine
from transitions.extensions import GraphMachine

states = ['Sunny', 'Cloudy', 'Rain']#,'Snow']

transitions = []
for i in range(3):
    for j in range(3):
      d={}
      d['trigger']=str(round(weather_markov[i][j], 2))
      d['source']=states[i]
      d['dest']=states[j]
      transitions.append(d)
print(transitions)

class Model(object):
    pass
model = Model()
machine = GraphMachine(model = model, states=states, transitions=transitions, initial=states[0], auto_transitions=False)
model.get_graph().draw('transitions.png', prog='dot', format='png') #ファイルで出力

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[{'trigger': '0.6', 'source': 'Sunny', 'dest': 'Sunny'}, {'trigger': '0.37', 'source': 'Sunny', 'dest': 'Cloudy'}, {'trigger': '0.03', 'source': 'Sunny', 'dest': 'Rain'}, {'trigger': '0.34', 'source': 'Cloudy', 'dest': 'Sunny'}, {'trigger': '0.48', 'source': 'Cloudy', 'dest': 'Cloudy'}, {'trigger': '0.19', 'source': 'Cloudy', 'dest': 'Rain'}, {'trigger': '0.32', 'source': 'Rain', 'dest': 'Sunny'}, {'trigger': '0.45', 'source': 'Rain', 'dest': 'Cloudy'}, {'trigger': '0.23', 'source': 'Rain', 'dest': 'Rain'}]
